In [ ]:
from script.requirements import *
from datetime import date as data_
pd.set_option('display.max_columns', None)

In [ ]:
df_materiali = pd.read_csv('data/materiali.csv', delimiter = ',', keep_default_na=False, dtype = {'MATNR' : str, 'LIFNR' : str})
df_anagraf = pd.read_csv('data/P0002.csv', delimiter = ',', keep_default_na=False, dtype = {'PERNR' : str})

# Lista materiali
material_list = df_materiali['MATNR'].apply(int)

In [ ]:
# Elenco fatture per evitare duplicati
invoices = np.array([])
orders = np.array([])
start_date = pd.to_datetime('2019-01-01')
end_date = start_date + dt.timedelta(days = 1517)

# Ordini da generare
n_PR = 5000

# Array date richieste per metterle in ordine
date_purchase_order = []
for o in range(n_PR):
    date_purchase_order.append(random_dates_business(start_date, end_date))
date_purchase_order = np.sort(date_purchase_order)

last_PR = 1000001

In [ ]:
%run -i "script/data_generation.py"

In [ ]:
invoice_payment = (data_.today() - dt.timedelta(days=7)).strftime('%Y-%m-%d')

In [ ]:
EKKO = pd.DataFrame(df[['WERKS', 'EBELN', 'AEDAT']]).drop_duplicates().reset_index(drop=True)

EKPO = pd.DataFrame(df[['WERKS', 'EBELN','EBELP',
                        'MATNR', 'MATKL', 'LIFNR', 'RKDST', 'MENGE', 'DELGE', 'MEINS', 'NETPR', 'PEINH',
                        'DRDAT', 'LEWED', 'SEDAT', 'LCWED','ITDAT',
                        'LOEKE', 'ELIKZ', 'EKNAM', 'EKRNR',
                        'BANFN', 'BNFPO',
                        'BELNR', 'BUZEI']]).drop_duplicates().reset_index(drop=True)

EBAN = pd.DataFrame(df[['WERKS', 'BANFN', 'BNFPO',
                        'MATNR', 'MATKL', 'MENGE', 'MEINS', 'NETPR', 'PEINH',
                        'BADAT', 'FRGDT',
                        'LOEKZ', 'ERNAM', 'PERNR',
                        'EBELN', 'EBELP']])
EBAN = EBAN.drop(EBAN.loc[EBAN['BADAT'].isnull()].index).reset_index(drop=True)

RBKP = pd.DataFrame(df[['WERKS', 'BELNR',
                        'BLDAT','BUDAT','BPDAT',
                        'EREKZ','LIFNR']]).drop_duplicates().reset_index(drop=True)

RBKP['BLOCK'] = pd.NaT
RBKP['RLOCK'] = pd.NaT

RSEG = pd.DataFrame(df[['WERKS', 'BELNR','BUZEI',
                        'MATNR', 'MATKL', 'MENGE', 'MEINS', 'NETPR', 'PEINH',
                        'EBELN','EBELP']]).reset_index(drop=True)

MARA = pd.DataFrame(df_materiali[['MATNR', 'MATKL' ,'MEINS', 'NETPR','PEINH','RKDST', 'PLIFZ', 'LIFNR', 'NAME1', 'PLICZ']]).reset_index(drop=True)

P0002 = pd.DataFrame(df_anagraf[['PERNR', 'ERNAM', 'NAME2', 'NACHN', 'EMAIL', 'WERKS', 'MATKL', 'USERT']]).reset_index(drop=True)
P0002['ERNAM'] = P0002['ERNAM'].astype(str)

EKDS = pd.DataFrame(columns = ['EBELN', 'EBELP', 'UDATE', 'FIELD', 'VALUE'])

EBAN.loc[EBAN['WERKS'] == 'BR10', 'LAND1'] = 'BR'
EBAN.loc[EBAN['WERKS'] == 'IN10', 'LAND1'] = 'IN'
EBAN.loc[EBAN['WERKS'] == 'US10', 'LAND1'] = 'US'
EBAN.loc[EBAN['WERKS'] == 'JP11', 'LAND1'] = 'JP'
EBAN.loc[EBAN['WERKS'] == 'DE10', 'LAND1'] = 'DE'
EBAN.loc[EBAN['WERKS'] == 'CN10', 'LAND1'] = 'CN'

In [ ]:
EKPO.to_csv('data/latest_tab/EKPO.csv', index=False)
RBKP.to_csv('data/latest_tab/RBKP.csv', index=False)
EKKO.to_csv('data/latest_tab/EKKO.csv', index=False)
RSEG.to_csv('data/latest_tab/RSEG.csv', index=False)
EBAN.to_csv('data/latest_tab/EBAN.csv', index=False)
EKDS.to_csv('data/latest_tab/EKDS.csv', index=False)

In [ ]:
EBAN, RBKP, RSEG, EKPO, EKKO, _, _, EKDS = get_tables(anomaly = False, new = False)
%run -i "script/script_errori.py"

RBKP.loc[RBKP.BLDAT > invoice_payment, ['BPDAT', 'EREKZ']] = np.NaN

#EBAN_0, RBKP_0, RSEG_0, EKPO_0, EKKO_0, _, _, EKDS_0 = get_tables(anomaly = False, new = False)
EBAN.to_csv('data/latest_tab/EBAN.csv', index=False)
EKPO.to_csv('data/latest_tab/EKPO.csv', index=False)
RSEG.to_csv('data/latest_tab/RSEG.csv', index=False)
EKDS.to_csv('data/latest_tab/EKDS.csv', index=False)
RBKP.to_csv('data/latest_tab/RBKP.csv', index=False)
EKKO.to_csv('data/latest_tab/EKKO.csv', index=False)

In [ ]:
today = (date.today() - dt.timedelta(days=1)).strftime('%Y-%m-%d')
EBAN.loc[EBAN.BADAT > today, 'BADAT'] = pd.NaT
EBAN.loc[EBAN.FRGDT > today, 'FRGDT'] = pd.NaT

EKKO.loc[EKKO.AEDAT > today, 'AEDAT'] = pd.NaT
EKPO.loc[EKPO.ITDAT > today, 'ELIKZ'] = ' '
EKPO.loc[EKPO.DRDAT > today, 'DRDAT'] = pd.NaT
EKPO.loc[EKPO.LEWED > today, 'LEWED'] = pd.NaT
EKPO.loc[EKPO.SEDAT > today, 'SEDAT'] = pd.NaT
EKPO.loc[EKPO.LCWED > today, 'LCWED'] = pd.NaT
EKPO.loc[EKPO.ITDAT > today, 'ITDAT'] = pd.NaT

RBKP.loc[RBKP.BPDAT > today, 'EREKZ'] = ' '
RBKP.loc[RBKP.BLDAT > today, ['BLDAT', 'BUDAT']] = pd.NaT
# RBKP.loc[RBKP.BUDAT > today, 'BUDAT'] = pd.NaT
RBKP.loc[RBKP.BPDAT > today, 'BPDAT'] = pd.NaT
RBKP.loc[RBKP.BLOCK > today, 'BLOCK'] = pd.NaT
RBKP.loc[RBKP.RLOCK > today, 'RLOCK'] = pd.NaT

EKDS.loc[EKDS.UDATE > today, 'UDATE'] = pd.NaT

In [ ]:
data_pool.create_table(df=EKPO, table_name="EKPO", drop_if_exists=True, force = True)
data_pool.create_table(df=RSEG, table_name="RSEG", drop_if_exists=True, force = True)
data_pool.create_table(df=EBAN, table_name="EBAN", drop_if_exists=True, force = True)
data_pool.create_table(df=EKDS, table_name="EKDS", drop_if_exists=True, force = True)
data_pool.create_table(df=RBKP, table_name="RBKP", drop_if_exists=True, force = True)
data_pool.create_table(df=EKKO, table_name="EKKO", drop_if_exists=True, force = True)